# 🤖 NexoCommerce - Treinamento e Avaliação do Modelo ML

**Objetivo:** Treinar e avaliar o modelo de predição de ciclo de vida de produtos

**Conteúdo:**
1. Carregamento dos dados processados
2. Divisão treino/validação/teste
3. Treinamento do modelo
4. Avaliação de performance
5. Análise de features importantes
6. Otimização de hiperparâmetros
7. Salvamento do modelo

---

In [1]:
# Imports
import sys
import os
from pathlib import Path

# Add src to path
sys.path.append(str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    accuracy_score, precision_score, recall_score, f1_score
)

import warnings
warnings.filterwarnings('ignore')

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Imports realizados com sucesso!")

✅ Imports realizados com sucesso!


## 1️⃣ Carregamento e Preparação dos Dados

In [2]:
from src.pipeline.data_loader import DataLoader
from src.pipeline.data_processing import DataProcessor

# Carregar dados
loader = DataLoader()
df = loader.load_data(source="synthetic", n_samples=2000)

print(f"✅ Dados carregados: {len(df)} produtos")
print(f"📊 Shape: {df.shape}")

INFO:src.pipeline.data_loader:Generating 2000 synthetic samples
INFO:src.pipeline.data_loader:Generated synthetic data with shape: (2000, 17)


✅ Dados carregados: 2000 produtos
📊 Shape: (2000, 17)


In [3]:
# Processar dados
processor = DataProcessor()
processed_df, features = processor.process_pipeline(df, is_training=True)

print(f"✅ Features criadas: {len(features)}")
print(f"\n📋 Features utilizadas:")
for i, feat in enumerate(features, 1):
    print(f"{i:2d}. {feat}")

INFO:src.pipeline.data_processing:Creating engineered features...
INFO:src.pipeline.data_processing:Created 29 total features
INFO:src.pipeline.data_processing:Creating target variable...
INFO:src.pipeline.data_processing:Target distribution:
lifecycle_action
0      75
1    1499
2     426
Name: count, dtype: int64
INFO:src.pipeline.data_processing:Target percentages:
lifecycle_action
0     3.75
1    74.95
2    21.30
Name: count, dtype: float64
INFO:src.pipeline.data_processing:Preparing features for modeling...
INFO:src.pipeline.data_processing:Selected 22 features for modeling
INFO:src.pipeline.data_processing:Fitted and transformed features
INFO:src.pipeline.data_processing:Processing pipeline completed


✅ Features criadas: 22

📋 Features utilizadas:
 1. price
 2. stock_quantity
 3. sales_last_30d
 4. views_last_30d
 5. rating
 6. num_reviews
 7. days_since_launch
 8. discount_percentage
 9. return_rate
10. conversion_rate
11. stock_coverage_days
12. revenue_last_30d
13. review_engagement_rate
14. price_competitiveness
15. performance_score
16. sales_velocity
17. discount_impact
18. rating_quality
19. category_encoded
20. age_category_encoded
21. stock_status_encoded
22. price_tier_encoded


In [4]:
# Separar features e target
X = processed_df[features]
y = processed_df['lifecycle_action']

print(f"\n📊 Dataset preparado:")
print(f"   • Features (X): {X.shape}")
print(f"   • Target (y): {y.shape}")
print(f"\n🎯 Distribuição do target:")
print(y.value_counts())
print(f"\n⚖️ Balanceamento: {y.value_counts().min() / y.value_counts().max():.2%}")


📊 Dataset preparado:
   • Features (X): (2000, 22)
   • Target (y): (2000,)

🎯 Distribuição do target:
lifecycle_action
1    1499
2     426
0      75
Name: count, dtype: int64

⚖️ Balanceamento: 5.00%


## 2️⃣ Divisão dos Dados (Train/Val/Test)

In [5]:
# Divisão estratificada: 70% treino, 15% validação, 15% teste
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp  # 0.176 * 0.85 ≈ 0.15
)

print("📊 Divisão dos dados:")
print(f"   • Treino:     {len(X_train):4d} amostras ({len(X_train)/len(X)*100:.1f}%)")
print(f"   • Validação:  {len(X_val):4d} amostras ({len(X_val)/len(X)*100:.1f}%)")
print(f"   • Teste:      {len(X_test):4d} amostras ({len(X_test)/len(X)*100:.1f}%)")

print("\n🎯 Distribuição do target em cada conjunto:")
print("\nTreino:")
print(y_train.value_counts())
print("\nValidação:")
print(y_val.value_counts())
print("\nTeste:")
print(y_test.value_counts())

📊 Divisão dos dados:
   • Treino:     1400 amostras (70.0%)
   • Validação:   300 amostras (15.0%)
   • Teste:       300 amostras (15.0%)

🎯 Distribuição do target em cada conjunto:

Treino:
lifecycle_action
1    1049
2     298
0      53
Name: count, dtype: int64

Validação:
lifecycle_action
1    225
2     64
0     11
Name: count, dtype: int64

Teste:
lifecycle_action
1    225
2     64
0     11
Name: count, dtype: int64


## 3️⃣ Treinamento do Modelo

In [6]:
from src.models.product_model import ProductLifecycleModel
import time

# Inicializar modelo
model = ProductLifecycleModel(model_type="random_forest")

print("🚀 Iniciando treinamento do modelo...")
print(f"   Modelo: {model.model_type}")
print(f"   Amostras de treino: {len(X_train)}")

# Treinar
start_time = time.time()
model.train(X_train, y_train, log_mlflow=False)
training_time = time.time() - start_time

print(f"\n✅ Treinamento concluído em {training_time:.2f}s")

INFO:src.models.product_model:Training random_forest model...
INFO:src.models.product_model:Train size: 1120, Test size: 280
INFO:src.models.product_model:Model training completed
INFO:src.models.product_model:Training completed. Test Accuracy: 0.9679


🚀 Iniciando treinamento do modelo...
   Modelo: random_forest
   Amostras de treino: 1400

✅ Treinamento concluído em 0.17s


## 4️⃣ Avaliação no Conjunto de Validação

In [7]:
# Predições no conjunto de validação
val_predictions = model.predict(X_val)
val_predictions_proba = model.predict_proba(X_val)

# Métricas
val_accuracy = accuracy_score(y_val, val_predictions)
val_precision = precision_score(y_val, val_predictions, average='weighted')
val_recall = recall_score(y_val, val_predictions, average='weighted')
val_f1 = f1_score(y_val, val_predictions, average='weighted')

print("📊 MÉTRICAS NO CONJUNTO DE VALIDAÇÃO")
print("="*50)
print(f"   Accuracy:  {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")
print(f"   Precision: {val_precision:.4f}")
print(f"   Recall:    {val_recall:.4f}")
print(f"   F1-Score:  {val_f1:.4f}")
print("="*50)

📊 MÉTRICAS NO CONJUNTO DE VALIDAÇÃO
   Accuracy:  0.9633 (96.33%)
   Precision: 0.9650
   Recall:    0.9633
   F1-Score:  0.9560


In [8]:
# Classification Report
print("\n📋 CLASSIFICATION REPORT (Validação)")
print("="*70)
print(classification_report(y_val, val_predictions))


📋 CLASSIFICATION REPORT (Validação)
              precision    recall  f1-score   support

           0       1.00      0.27      0.43        11
           1       0.95      1.00      0.98       225
           2       1.00      0.95      0.98        64

    accuracy                           0.96       300
   macro avg       0.98      0.74      0.79       300
weighted avg       0.97      0.96      0.96       300



In [9]:
# Confusion Matrix
cm = confusion_matrix(y_val, val_predictions)
labels = sorted(y_val.unique())

fig = px.imshow(
    cm,
    labels=dict(x="Predito", y="Real", color="Contagem"),
    x=labels,
    y=labels,
    text_auto=True,
    color_continuous_scale='Blues',
    title='🎯 Matriz de Confusão (Validação)'
)
fig.update_layout(height=500, width=600)
fig.show()

# Calcular acurácia por classe
print("\n📊 Acurácia por classe:")
for i, label in enumerate(labels):
    class_accuracy = cm[i, i] / cm[i, :].sum()
    print(f"   {label}: {class_accuracy:.2%}")


📊 Acurácia por classe:
   0: 27.27%
   1: 100.00%
   2: 95.31%


## 5️⃣ Feature Importance

In [13]:
# ============================================================================
# Obter importância das features
# ============================================================================

feature_importance = model.get_feature_importance()

if feature_importance is not None:
    # Converter para array numpy se for DataFrame ou Series
    if isinstance(feature_importance, pd.DataFrame):
        feature_importance = feature_importance.values.flatten()
    elif isinstance(feature_importance, pd.Series):
        feature_importance = feature_importance.values
    elif hasattr(feature_importance, 'ndim') and feature_importance.ndim > 1:
        feature_importance = feature_importance.flatten()
    
    # Verificar se o tamanho corresponde
    if len(feature_importance) != len(features):
        print(f"⚠️ Aviso: Tamanho de feature_importance ({len(feature_importance)}) != features ({len(features)})")
        # Ajustar para o menor tamanho
        min_len = min(len(feature_importance), len(features))
        feature_importance = feature_importance[:min_len]
        features_to_use = features[:min_len]
    else:
        features_to_use = features
    
    # Garantir que feature_importance seja numérico
    try:
        feature_importance = pd.to_numeric(feature_importance, errors='coerce')
    except:
        pass
    
    # Criar DataFrame
    importance_df = pd.DataFrame({
        'feature': features_to_use,
        'importance': feature_importance
    })
    
    # Remover valores NaN se houver
    importance_df = importance_df.dropna(subset=['importance'])
    
    # Ordenar
    importance_df = importance_df.sort_values('importance', ascending=False)
    
    print("🔝 TOP 15 Features Mais Importantes:")
    print("="*50)
    display(importance_df.head(15))
    
    # Visualização
    fig = px.bar(
        importance_df.head(20),
        x='importance',
        y='feature',
        orientation='h',
        title='📊 Top 20 Features Mais Importantes',
        labels={'importance': 'Importância', 'feature': 'Feature'},
        color='importance',
        color_continuous_scale='Viridis'
    )
    fig.update_layout(height=600, showlegend=False)
    fig.show()
    
    # Importância acumulada
    importance_df['cumulative_importance'] = importance_df['importance'].cumsum()
    
    # Quantas features para 80% da importância?
    n_features_80 = (importance_df['cumulative_importance'] <= 0.8).sum()
    print(f"\n📌 {n_features_80} features explicam 80% da importância total")
else:
    print("⚠️ Feature importance não disponível para este modelo")

⚠️ Aviso: Tamanho de feature_importance (20) != features (22)
🔝 TOP 15 Features Mais Importantes:


,feature,importance
1,stock_quantity,0.314441
3,views_last_30d,0.251653
5,num_reviews,0.136331
7,discount_percentage,0.039404
9,conversion_rate,0.039296
11,revenue_last_30d,0.035410
13,price_competitiveness,0.030761
15,sales_velocity,0.024999
17,rating_quality,0.023769
19,age_category_encoded,0.021524



📌 5 features explicam 80% da importância total


## 7️⃣ Avaliação Final no Conjunto de Teste

In [15]:
# Predições no conjunto de teste
test_predictions = model.predict(X_test)
test_predictions_proba = model.predict_proba(X_test)

# Métricas
test_accuracy = accuracy_score(y_test, test_predictions)
test_precision = precision_score(y_test, test_predictions, average='weighted')
test_recall = recall_score(y_test, test_predictions, average='weighted')
test_f1 = f1_score(y_test, test_predictions, average='weighted')

print("🎯 MÉTRICAS FINAIS NO CONJUNTO DE TESTE")
print("="*50)
print(f"   Accuracy:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"   Precision: {test_precision:.4f}")
print(f"   Recall:    {test_recall:.4f}")
print(f"   F1-Score:  {test_f1:.4f}")
print("="*50)

🎯 MÉTRICAS FINAIS NO CONJUNTO DE TESTE
   Accuracy:  0.9533 (95.33%)
   Precision: 0.9552
   Recall:    0.9533
   F1-Score:  0.9480


In [16]:
# Classification Report (Teste)
print("\n📋 CLASSIFICATION REPORT (Teste)")
print("="*70)
print(classification_report(y_test, test_predictions))


📋 CLASSIFICATION REPORT (Teste)
              precision    recall  f1-score   support

           0       1.00      0.36      0.53        11
           1       0.95      1.00      0.97       225
           2       0.98      0.91      0.94        64

    accuracy                           0.95       300
   macro avg       0.98      0.76      0.82       300
weighted avg       0.96      0.95      0.95       300



In [17]:
# Confusion Matrix (Teste)
cm_test = confusion_matrix(y_test, test_predictions)

fig = px.imshow(
    cm_test,
    labels=dict(x="Predito", y="Real", color="Contagem"),
    x=labels,
    y=labels,
    text_auto=True,
    color_continuous_scale='Greens',
    title='🎯 Matriz de Confusão (Teste Final)'
)
fig.update_layout(height=500, width=600)
fig.show()

## 8️⃣ Análise de Confiança das Predições

In [18]:
# Calcular confiança (max probability)
test_confidence = test_predictions_proba.max(axis=1)

# Criar DataFrame com resultados
results_df = pd.DataFrame({
    'real': y_test.values,
    'predicted': test_predictions,
    'confidence': test_confidence,
    'correct': y_test.values == test_predictions
})

print("📊 ANÁLISE DE CONFIANÇA")
print("="*50)
print(f"   Confiança média:    {test_confidence.mean():.4f}")
print(f"   Confiança mediana:  {np.median(test_confidence):.4f}")
print(f"   Confiança mínima:   {test_confidence.min():.4f}")
print(f"   Confiança máxima:   {test_confidence.max():.4f}")
print("="*50)

# Distribuição de confiança
fig = px.histogram(
    results_df,
    x='confidence',
    color='correct',
    nbins=50,
    title='📊 Distribuição de Confiança (Corretas vs Incorretas)',
    labels={'confidence': 'Confiança', 'correct': 'Predição Correta'},
    barmode='overlay',
    opacity=0.7
)
fig.update_layout(height=500)
fig.show()

# Acurácia por faixa de confiança
confidence_bins = pd.cut(results_df['confidence'], bins=[0, 0.5, 0.7, 0.9, 1.0])
accuracy_by_confidence = results_df.groupby(confidence_bins)['correct'].mean()

print("\n📊 Acurácia por faixa de confiança:")
print(accuracy_by_confidence)

📊 ANÁLISE DE CONFIANÇA
   Confiança média:    0.9172
   Confiança mediana:  0.9607
   Confiança mínima:   0.5026
   Confiança máxima:   1.0000



📊 Acurácia por faixa de confiança:
confidence
(0.0, 0.5]     NaN
(0.5, 0.7]    0.44
(0.7, 0.9]    1.00
(0.9, 1.0]    1.00
Name: correct, dtype: float64


## 9️⃣ Comparação: Random Forest vs Gradient Boosting

In [19]:
# Treinar Gradient Boosting
print("🚀 Treinando Gradient Boosting para comparação...")

model_gb = ProductLifecycleModel(model_type="gradient_boosting")
model_gb.train(X_train, y_train, log_mlflow=False)

# Predições
gb_predictions = model_gb.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_predictions)
gb_f1 = f1_score(y_test, gb_predictions, average='weighted')

print("\n📊 COMPARAÇÃO DE MODELOS")
print("="*60)
print(f"{'Modelo':<25} {'Accuracy':<15} {'F1-Score':<15}")
print("="*60)
print(f"{'Random Forest':<25} {test_accuracy:<15.4f} {test_f1:<15.4f}")
print(f"{'Gradient Boosting':<25} {gb_accuracy:<15.4f} {gb_f1:<15.4f}")
print("="*60)

# Visualização
comparison_df = pd.DataFrame({
    'Model': ['Random Forest', 'Gradient Boosting'],
    'Accuracy': [test_accuracy, gb_accuracy],
    'F1-Score': [test_f1, gb_f1]
})

fig = go.Figure()
fig.add_trace(go.Bar(
    name='Accuracy',
    x=comparison_df['Model'],
    y=comparison_df['Accuracy'],
    text=comparison_df['Accuracy'].round(4),
    textposition='auto'
))
fig.add_trace(go.Bar(
    name='F1-Score',
    x=comparison_df['Model'],
    y=comparison_df['F1-Score'],
    text=comparison_df['F1-Score'].round(4),
    textposition='auto'
))
fig.update_layout(
    title='📊 Comparação de Performance dos Modelos',
    yaxis_title='Score',
    barmode='group',
    height=500
)
fig.show()

# Escolher melhor modelo
best_model = model if test_accuracy >= gb_accuracy else model_gb
best_model_name = "Random Forest" if test_accuracy >= gb_accuracy else "Gradient Boosting"
print(f"\n🏆 Melhor modelo: {best_model_name}")

INFO:src.models.product_model:Training gradient_boosting model...
INFO:src.models.product_model:Train size: 1120, Test size: 280


🚀 Treinando Gradient Boosting para comparação...


INFO:src.models.product_model:Model training completed
INFO:src.models.product_model:Training completed. Test Accuracy: 0.9929



📊 COMPARAÇÃO DE MODELOS
Modelo                    Accuracy        F1-Score       
Random Forest             0.9533          0.9480         
Gradient Boosting         0.9900          0.9901         



🏆 Melhor modelo: Gradient Boosting


## 🔟 Salvamento do Modelo

In [ ]:
# Salvar o melhor modelo
model_path = Path.cwd().parent / 'models' / 'product_lifecycle_model.pkl'
model_path.parent.mkdir(parents=True, exist_ok=True)

best_model.save_model(str(model_path))

print(f"✅ Modelo salvo em: {model_path}")
print(f"   Tipo: {best_model_name}")
print(f"   Accuracy: {test_accuracy:.4f}")
print(f"   F1-Score: {test_f1:.4f}")

## 📊 Resumo Final

In [20]:
print("="*80)
print("🎯 RESUMO DO TREINAMENTO DO MODELO")
print("="*80)

print(f"\n1️⃣ DADOS:")
print(f"   • Total de amostras: {len(X):,}")
print(f"   • Features: {len(features)}")
print(f"   • Classes: {len(labels)}")
print(f"   • Treino/Val/Teste: {len(X_train)}/{len(X_val)}/{len(X_test)}")

print(f"\n2️⃣ MODELO:")
print(f"   • Tipo: {best_model_name}")
print(f"   • Tempo de treino: {training_time:.2f}s")

print(f"\n3️⃣ PERFORMANCE (Teste):")
print(f"   • Accuracy:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"   • Precision: {test_precision:.4f}")
print(f"   • Recall:    {test_recall:.4f}")
print(f"   • F1-Score:  {test_f1:.4f}")

print(f"\n4️⃣ CROSS-VALIDATION:")
print(f"   • Média: {cv_scores.mean():.4f}")
print(f"   • Std:   {cv_scores.std():.4f}")

print(f"\n5️⃣ CONFIANÇA:")
print(f"   • Média: {test_confidence.mean():.4f}")
print(f"   • Mediana: {np.median(test_confidence):.4f}")

if feature_importance is not None:
    top_3_features = importance_df.head(3)['feature'].tolist()
    print(f"\n6️⃣ TOP 3 FEATURES:")
    for i, feat in enumerate(top_3_features, 1):
        print(f"   {i}. {feat}")

print("\n" + "="*80)
print("✅ Treinamento e avaliação concluídos!")
print("📌 Próximo passo: Demonstração do sistema multi-agente (notebook 03)")
print("="*80)

🎯 RESUMO DO TREINAMENTO DO MODELO

1️⃣ DADOS:
   • Total de amostras: 2,000
   • Features: 22
   • Classes: 3
   • Treino/Val/Teste: 1400/300/300

2️⃣ MODELO:
   • Tipo: Gradient Boosting
   • Tempo de treino: 0.17s

3️⃣ PERFORMANCE (Teste):
   • Accuracy:  0.9533 (95.33%)
   • Precision: 0.9552
   • Recall:    0.9533
   • F1-Score:  0.9480

4️⃣ CROSS-VALIDATION:
   • Média: 0.9600
   • Std:   0.0094

5️⃣ CONFIANÇA:
   • Média: 0.9172
   • Mediana: 0.9607

6️⃣ TOP 3 FEATURES:
   1. stock_quantity
   2. views_last_30d
   3. num_reviews

✅ Treinamento e avaliação concluídos!
📌 Próximo passo: Demonstração do sistema multi-agente (notebook 03)
